<a href="https://colab.research.google.com/github/pszemraj/job-scraper/blob/master/switzerland/Job_Scraper_CH_Edition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Job Scraping + Viz: Modified for CH

---
Created by: [Peter Szemraj](https://peterszemraj.ch/)

1. [link](https://github.com/pszemraj/job-scraper) to the repo of this project

*Date 07.07.2021*

---
original code forked from [this article](https://towardsdatascience.com/automating-my-job-search-with-python-ee2b465c6a8f) on medium

Key Features:
1. connects to your google drive and saves the search files there
2. clusters and visualizes job postings for faster "find what I actually care about"


**Note**

If you are editing this code / adapting to a different country/site for Indeed, sections that likely need to be changed are described with comments <font color='orange'>**in orange font**</font>

---


# Setup

### Google Drive

- If you don't wish to use this part, you can skip it. The below should still work on the virtual machine local directory, but files will likely need to be downloaded manually
- To manually download files, click on the folder on the left side of your colab window (in the navigation pane)


In [1]:
from datetime import date
from google.colab import files
from google.colab import drive
import os
from os.path import join

### Filepaths on Drive

saves all runs to a folder in your google drive with today's date. Folder is created if it doesn't exist (run cell below)

<font color='orange'> update the **directory** variable to a custom filepath if desired (only relevant if using google drive) </font>

In [2]:
use_google_drive = False #@param {type:"boolean"}
gdrive_directory = "/content/drive/MyDrive/Career/job_search_repo" #@param {type:"string"}
os.makedirs(gdrive_directory, exist_ok=True)
today = date.today()
# Month abbreviation, day and year	
td_date = today.strftime("_%b-%d-%Y_")
folder_name = "job_search_results_" + td_date

output_folder_path = join(gdrive_directory, folder_name)

# create interface to upload / interact with google drive and video files
if use_google_drive:
    drive.mount('/content/drive')
    os.makedirs(gdrive_directory, exist_ok=True)
    os.chdir(output_folder_path)
else:
    output_folder_path = os.getcwd()
    print("Not using google drive. will save files to google colab VM at {}".format(output_folder_path))
# after you allow the authentication, you can work using the path "/content/drive/My Drive"
# if it works it will say "Mounted at /content/drive"

Not using google drive. will save files to google colab VM at /content


### Install & Import


In [3]:
%%capture
"""
libraries added / installed by Peter

"""

!pip install -U selenium
!pip install -U bs4
!pip install -U texthero
!pip install -U plotly
!pip install pyshorteners
!pip install -U kneed
!pip install -U tensorflow_hub

from os.path import join
from google.colab import data_table
from datetime import datetime, date
import plotly.express as px
import texthero as hero
import pprint as pp
import time

%load_ext google.colab.data_table


In [4]:
"""
libraries imported in the originally forked version of this code. 

Created on Tue Apr 28 11:35:04 2020
@author: chrislovejoy
"""
import urllib
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import os

formats jupyter notebook / colab

In [5]:
from IPython.display import HTML, display
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# Job Search Functions


## generic

cuttly [website](https://cutt.ly/edit)
cuttly tutorial w/ Python [here](https://towardsdatascience.com/best-apis-for-url-shortening-using-python-2db09d1f86f0)


pyshorteners [docs](https://pyshorteners.readthedocs.io/en/latest/)


### misc.

<font color='orange'> add own API key for link shortening as needed</font>


In [6]:
import requests
import time, random
import pyshorteners

def save_jobs_to_excel(jobs_list, filename, verbose=False):
    # i have no idea what this function does
    jobs = pd.DataFrame(jobs_list)
    jobs.to_excel(filename)

    if verbose:
        print("saved the following to excel with filename {}: \n".format(filename))

        print(jobs.info())
    return jobs

def shorten_URL_bitly(long_url, verbose=False):
    # requires free account / API token. https://bitly.com/
    # generate short URLs from the ones scraped 

    time.sleep(random.randint(1,5)) # don't overload API

    ACCESS_TOKEN = "hahah_get_ur_own"

    # Shorten long URL
    try:
        s = pyshorteners.Shortener(api_key=ACCESS_TOKEN)
        short_url = s.bitly.short(long_url)

        if verbose:
            print ("Short URL is {}".format(short_url))
    except:
        print("Error accessing API for key {} and fn shorten_URL_bitly".format(ACCESS_TOKEN))
        print("Try updating API key / checking fn. Returning original url")
        short_url = long_url
        
    return short_url


def text_first_N(text, num=40):
    # returns the first N chars in text, i.e. for long job descriptions
    # for use with Pandas .apply() function

    text = str(text) #convert to string

    if isinstance(text, list):
        text = " ".join(text)

    if len(text) <= num:
        return text
    else:
        short_text = text[:num]
        return short_text + ".."

### optimize k-means no.

stole this snippet from a different project of mine - was written several months ago, don't judge the efficiency

In [7]:
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

def optimal_num_clustas(input_matrix, d_title, top_end=11, show_plot=False,
                        write_image=False):
    # given 'input_matrix' as a pandas series containing a list / vector in each
    # row, find the optimal number of k_means clusters to cluster them using 
    # the elbow method

    # 'top_end' is the max number of clusters. If having issues, look at the plot
    # and adjust accordingly

    scaler = StandardScaler()
    # texthero input data structure is weird.
    #  stole the below if/else from the source code behind TH kmeans fn
    # https://github.com/jbesomi/texthero/blob/master/texthero/representation.py

    if isinstance(input_matrix, pd.DataFrame):
        # fixes weird issues parsing a texthero edited text pd series
        input_matrix_coo = input_matrix.sparse.to_coo()
        input_matrix_for_vectorization = input_matrix_coo.astype("float64")
    else:
        input_matrix_for_vectorization = list(input_matrix)

    scaled_features = scaler.fit_transform(input_matrix_for_vectorization)
    kmeans_kwargs = {
        "init": "random",
        "n_init": 30,
        "max_iter": 300,
        "random_state": 42
    }
    # A list holds the SSE values for each k
    sse = []
    for k in range(1, top_end):
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(scaled_features)
        sse.append(kmeans.inertia_)

    # plot to illustrate (viewing it is optional)
    title_k = 'Optimal k-means for' + d_title
    kmeans_opt_df = pd.DataFrame(list(zip(range(1, top_end), sse)), columns=['Number of Clusters', 'SSE'])
    f_k = px.line(kmeans_opt_df, x='Number of Clusters', y='SSE', title=title_k)
    # find optimum
    kl = KneeLocator(range(1, top_end), sse, 
                     curve="convex", direction="decreasing")
    onk = kl.elbow

    if onk is None:
        print("Warning - {} has no solution for optimal k-means".format(d_title))
        print("Returning # of clusters as max allowed ( {} )".format(top_end))
        return top_end

    if onk == top_end:
        print("Warning - {} opt. # equals max value searched ({})".format(d_title,
                                                                          top_end))

    print("\nFor {}: opt. # of k-means clusters is {} \n".format(d_title, onk))
    f_k.add_vline(x=onk) # add vertical line to plotly 

    if show_plot:
        f_k.show() 

    if write_image:
        f_k.write_image(join(output_path_full, title_k + ".png"))

    return onk

---

## visualization 

Use texthero built-in visualization methods and then plot them *explicitly* in Plotly because it has dark mode and TextHero's plotly wrapper function does not

- texthero represendation [docs](https://texthero.org/docs/api-representation)

- plotly [scatterplot overview](https://plotly.com/python/line-and-scatter/)
    - px.scatter() detailed [documentation](https://plotly.com/python-api-reference/generated/plotly.express.scatter)

### Options:
1. ```viz_job_data(df)``` is the OG version, vectorizes text with TFIDF and plots with PCA. 
    - Uses k-means nclusters as 5
2. ```viz_job_data_word2vec(df)``` vectorizes text with word2vec (*computing "average" 300-d vector*) and plots with PCA
    - computes optimal # of kmeans clusters (max set to 15)

---

### standard


uses everything built-in to the texthero package:
- keeps kmeans clusters @ 5
- text is converted to vector based on tf-idf
    - (anecdotally this doesn't seem to work well if your "document" is short like these job listings)

In [8]:
from datetime import datetime, date
import plotly.express as px

plots_made = 0

def viz_job_data(viz_df, text_col_name, save_plot=False, h=720):

    today = date.today()
    # Month abbreviation, day and year	
    td_str = today.strftime("%b-%d-%Y")

    viz_df['tfidf'] = (
        viz_df[text_col_name]
        .pipe(hero.clean)
        .pipe(hero.tfidf)
    )

    viz_df['kmeans'] = (
        viz_df['tfidf']
        .pipe(hero.kmeans, n_clusters=5)
        .astype(str)
    )

    viz_df['pca'] = (
        viz_df['tfidf']
        .pipe(hero.pca)
    )

    hv_list = list(viz_df.columns)
    hv_list.remove('tfidf')
    hv_list.remove('pca')
    hv_list.remove('summary')

    plot_title = td_str + " Vizualize Companies by {} Data".format(text_col_name)

    # reformat data so don't have to use built-in plotting 

    df_split_pca = pd.DataFrame(viz_df["pca"].to_list(), columns=['pca_x', 'pca_y'])
    viz_df.drop(columns="pca", inplace=True) # drop original PCA column
    viz_df = pd.concat([viz_df, df_split_pca], axis=1) # merge dataframes

    # plot pca data
    # texthero also features pther ways to reduce dimensions besides pca, see docs

    w = int(h * (4/3))

    fig_s = px.scatter(viz_df, x= "pca_x", y="pca_y", color="kmeans", 
                     hover_data=hv_list, title=plot_title, height=h, width=w,
                     template="plotly_dark")
    fig_s.show()

    if save_plot:
        plots_made += 1

        fig.write_html(plot_title + str(plots_made) + ".html", 
                       include_plotlyjs=True)

    print("plot generated - ", datetime.now())

### word2vec viz

- word2vec [explanation](https://towardsdatascience.com/introduction-to-word-embedding-and-word2vec-652d0c2060fa) for those uninitiated
-gensim [link](https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html)

Re: using word2vec in a basic way for sentence embedding

> With the Word2Vec model, we can calculate the vectors for each word in a document. But what if we want to calculate a vector for the entire document? We could average the vectors for each word in the document - while this is quick and crude, it can often be useful. However, there is a better way… 
> *goes on to discuss doc2vec which does not have pretrained models*

---

re: ^ I am not doing the better way. Load the word2vec-google-news-300 dataset from gensim, then iterate through all words in the input text, generate the vector for that word, then take the mean of those. Only words with length 3 or greater are considered. 

---
<font color='orange'>**DISCLAIMER: IT TAKES A BIT TO DOWNLOAD THE GENSIM / GOOGLE MODEL**</font>
- <font color='orange'>possible to save the model locally (on disk/gdrive) if desired</font>


In [9]:
import gensim.downloader as api

def load_gensim_word2vec(wvmodel="word2vec-google-news-300", verbose=False):
    # another option is the smaller: api.load("word2vec-ruscorpora-300")
    loaded_model = api.load(wvmodel)

    print("loaded data for word2vec - ", datetime.now())

    if verbose:
        # for more info or bug fixing
        wrdvecs = pd.DataFrame(loaded_model.vectors, 
                               index=loaded_model.key_to_index)
        print("created dataframe from word2vec data- ", datetime.now())
        print('dimensions of the df: \n', wrdvecs.shape)

    print("testing gensim model...")
    test_string = "computer"
    vector = loaded_model.wv[test_string]

    print("The shape of string {} is: \n {}".format(test_string, vector.shape))
    print("test complete - ", datetime.now())

    return loaded_model


iterate through all words in the input text generate the word2vec vector for that word, then take the mean of those. Only words with length 3 or greater are considered.

**note that if a word is not in the google news dataset, it is just skipped. If you think the graphs / representations are not good enough, you should check to ensure that a large amount of words (or fraction rather) are not being skipped)**
* the verbose parameter helps with that

In [10]:
import numpy as np

def get_vector_freetext(input_text, model, verbose=0, cutoff=2):

    # verbose = 1 shows you how many words were skipped
    # verbose = 2 tells you each individual skipped word and ^
    # 'cutoff' removes all words with length N or less from the rep. vector

    lower_it = input_text.lower()
    input_words = lower_it.split(" ") # yes, this is an assumption
    usable_words = [word for word in input_words if len(word) > cutoff]

    list_of_vectors = []
    num_words_total = len(usable_words)
    num_excluded = 0

    for word in usable_words:
        try:
            this_vector = model.wv[word]
            list_of_vectors.append(this_vector)
        except:
            num_excluded += 1
            if verbose == 2:
                print("\nThe word/term {} is not in the model vocab.".format(word))
                print("Excluding from representative vector")

    rep_vec = np.mean(list_of_vectors, axis=0)

    if verbose > 0:
        print("Computed representative vector. Excluded {} words out of {}".format(num_excluded,
                                                                            num_words_total))

    return rep_vec

#### Plotting function

In [11]:

def viz_job_data_word2vec(viz_df, text_col_name, save_plot=False, h=720,
                          query_name="", show_text=False):

    today = date.today()
    # Month abbreviation, day and year	
    td_str = today.strftime("%b-%d-%Y")

    # compute word2vec avg vector for each row of text
    viz_df['avg_vec'] = viz_df[text_col_name].apply(get_vector_freetext, 
                                                    args=(w2v_model,))

    # get optimal number of kmeans. limit max to 15 for interpretability
    max_clusters = 15
    if len(viz_df['avg_vec']) < max_clusters: max_clusters = len(viz_df['avg_vec'])

    kmeans_numC = optimal_num_clustas(viz_df['avg_vec'], 
                                      d_title='word2vec-' + query_name, 
                                      top_end=max_clusters)
    
    # complete k-means clustering + pca dim red. w/ avg_vec
    if kmeans_numC is None:
        kmeans_numC = 5 

    viz_df['kmeans'] = (
        viz_df['avg_vec']
        .pipe(hero.kmeans, n_clusters=kmeans_numC, 
              algorithm="elkan", random_state=42, n_init=30)
        .astype(str)
    )
    # texthero has other algs to reduce dimensions besides pca, see docs
    viz_df['pca'] = (
        viz_df['avg_vec']
        .pipe(hero.pca)
    )

    # generate list of column names for hover_data
    hv_list = list(viz_df.columns)
    hv_list.remove('avg_vec')
    hv_list.remove('pca')
    if "tfidf" in hv_list:
        hv_list.remove('tfidf')
    if "summary" in hv_list:
        hv_list.remove('summary')

    # reformat data so don't have to use texthero built-in plotting 
    df_split_pca = pd.DataFrame(viz_df["pca"].to_list(), 
                                columns=['pca_x', 'pca_y'])
    viz_df.drop(columns="pca", inplace=True) # drop original PCA column
    viz_df = pd.concat([viz_df, df_split_pca], axis=1) # merge dataframes

    # set up plot pars (width, title, text)
    w = int(h * (4/3))

    if len(query_name) > 0:
        # user provided query_name so include
        plot_title = td_str + " viz Jobs by '{}' via word2vec + pca".format(text_col_name) + " | " + query_name
    else:
        plot_title = td_str + " viz Jobs by '{}' via word2vec + pca".format(text_col_name)

    if show_text:
        # adds company names to the plot if you want
        viz_df["companies_abbrev"] = viz_df["companies"].apply(text_first_N, num=15)
        graph_text_label = "companies_abbrev"
    else:
        graph_text_label = None
    
    # plot dimension-reduced data
    fig_w2v = px.scatter(viz_df, x= "pca_x", y="pca_y", color="kmeans", 
                     hover_data=hv_list, title=plot_title, height=h, width=w,
                     template="plotly_dark", text=graph_text_label)
    fig_w2v.show()

    # save if requested 
    
    if save_plot:
        # saves the HTML file 
        # auto-saving as a static image is a lil difficult so just click on the interactive
        # plot it generates 
        fig_w2v.write_html(plot_title + query_name + "_" + text_col_name + ".html", 
                       include_plotlyjs=True)

    print("plot generated - ", datetime.now())

### google USE

**Google Universal Sentence Encoder**

```
# Load model from TensorFlow Site
# embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")  # Large
# embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-lite/2") # Lite
```

In [12]:
import tensorflow_hub as hub
 
def load_google_USE():
    st = time.time()
    embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
    rt = (time.time() - st) / 60
    print("loaded google USE embeddings in {} minutes".format(round(rt,2)))

    return embed


#### USE plotting fn

In [13]:

def vizjobs_googleUSE(viz_df, text_col_name, USE_embedding, save_plot=False, h=720,
                          query_name="", show_text=False, viz_type="TSNE"):

    today = date.today()
    # Month abbreviation, day and year	
    td_str = today.strftime("%b-%d-%Y")

    # generate embeddings for google USE. USE_embedding MUST be passed in
    embeddings = USE_embedding(viz_df[text_col_name])  # create list from np arrays
    use = np.array(embeddings).tolist()  # add lists as dataframe column
    viz_df['use_vec'] = use

    # get optimal number of kmeans. limit max to 15 for interpretability
    max_clusters = 15
    if len(viz_df['use_vec']) < max_clusters: max_clusters = len(viz_df['use_vec'])

    kmeans_numC = optimal_num_clustas(viz_df['use_vec'], 
                                      d_title='google_USE-' + query_name, 
                                      top_end=max_clusters)
    
    # complete k-means clustering + pca dim red. w/ use_vec
    if kmeans_numC is None:
        kmeans_numC = 5 

    viz_df['kmeans'] = (
        viz_df['use_vec']
        .pipe(hero.kmeans, n_clusters=kmeans_numC, 
              algorithm="elkan", random_state=42, n_init=30)
        .astype(str)
    )

    # use the vector for dimensionality reduction

    if viz_type.lower() == "tsne":
        viz_df['TSNE'] = (
                        viz_df['use_vec']
                                        .pipe(hero.tsne, random_state=42)
        )
    else:
        viz_df['pca'] = (
                        viz_df['use_vec']
                                        .pipe(hero.pca)
        )


    # generate list of column names for hover_data in the html plot

    hv_list = list(viz_df.columns)
    hv_list.remove('use_vec')

    if "tfidf" in hv_list:
        hv_list.remove('tfidf')
    if "pca" in hv_list:
        hv_list.remove('pca')
    if "TSNE" in hv_list:
        hv_list.remove('TSNE')
    if "summary" in hv_list:
        hv_list.remove('summary')

    # reformat data so don't have to use texthero built-in plotting 

    if viz_type.lower() == "tsne":
        # TSNE reformat
        df_split_tsne = pd.DataFrame(viz_df["TSNE"].to_list(), 
                                    columns=['tsne_x', 'tsne_y'])
        viz_df.drop(columns="TSNE", inplace=True) # drop original PCA column
        viz_df = pd.concat([viz_df, df_split_tsne], axis=1) # merge dataframes
    else:
        # PCA reformat
        df_split_pca = pd.DataFrame(viz_df["pca"].to_list(), 
                                    columns=['pca_x', 'pca_y'])
        viz_df.drop(columns="pca", inplace=True) # drop original PCA column
        viz_df = pd.concat([viz_df, df_split_pca], axis=1) # merge dataframes

    # set up plot pars (width, title, text)
    w = int(h * (4/3))

    if len(query_name) > 0:
        # user provided query_name so include
        plot_title = td_str + " viz Jobs by '{}' via google USE + {}".format(text_col_name, 
                                                                              viz_type) + " | " + query_name
    else:
        plot_title = td_str + " viz Jobs by '{}' via google USE {}".format(text_col_name,
                                                                              viz_type)
    if show_text:
        # adds company names to the plot if you want
        viz_df["companies_abbrev"] = viz_df["companies"].apply(text_first_N, num=15)
        graph_text_label = "companies_abbrev"
    else:
        graph_text_label = None

    # setup labels (decides pca or tsne)
    if viz_type.lower() == "tsne":
        plt_coords = ['tsne_x', 'tsne_y']
    else:
        plt_coords = ['pca_x', 'pca_y']
    
    # plot dimension-reduced data

    viz_df.dropna(inplace=True)

    fig_use = px.scatter(viz_df, x= plt_coords[0], y=plt_coords[1], color="kmeans", 
                     hover_data=hv_list, title=plot_title, height=h, width=w,
                     template="plotly_dark", text=graph_text_label)
    fig_use.show()

    # save if requested 
    
    if save_plot:
        # saves the HTML file 
        # auto-saving as a static image is a lil difficult so just click on the interactive
        # plot it generates 
        fig_use.write_html(plot_title + query_name + "_" + text_col_name + ".html", 
                       include_plotlyjs=True)

    print("plot generated - ", datetime.now())

---

---

## CH get_jobs() general fn

The main function find_jobs_from() adjusted to work for CH


<font color='orange'>if customizing this code, you need to change this</font>

In [14]:
from datetime import date
import pprint as pp

today = date.today()
# Month abbreviation, day and year	
d4 = today.strftime("%b-%d-%Y")

default_filename = d4 + "_[raw]_scraped_jobs_CH.xls"

def find_CHjobs_from(website, desired_characs, job_query, job_type=None, 
                   language=None, filename=default_filename, verbose=False):    
    """
    This function extracts all the desired characteristics of all new job postings
    of the title and location specified and returns them in single file.
    The arguments it takes are:
        - Website: to specify which website to search (options: 'indeed' or 'indeed_default')
        - job_query: words that you want to narrow down the jobs to. for example 'data'
        - job_type: 'internship' or 'fulltime' or 'permanent'
        - language: 'en' or 'de' or leave blank
        - Desired_characs: this is a list of the job characteristics of interest,
            from titles, companies, links and date_listed.
        - Filename: default is "JS_test_results.xls", can be changed to whatever 
    """
    
    if website == 'indeed':
        sp_search = load_indeed_jobs_CH(job_query, 
                                       job_type=job_type, language=language)
        job_soup = sp_search.get("job_soup")
        URL_used = sp_search.get("query_URL")

        if verbose:
            print("\n The full HTML docs are: \n")
            pp.pprint(job_soup, compact=True)
        jobs_list, num_listings = extract_job_information_indeedCH(job_soup, 
                                                                   desired_characs,
                                                                   uURL=URL_used)
    elif website == 'indeed_default':
        sp_search = load_indeed_jobs_CH(job_query, run_default=True)
        job_soup = sp_search.get("job_soup")
        URL_used = sp_search.get("query_URL")
        if verbose:
            print("\n The full HTML docs are: \n")
            pp.pprint(job_soup, compact=True)

        jobs_list, num_listings = extract_job_information_indeedCH(job_soup, 
                                                                   desired_characs,
                                                                   uURL=URL_used)

    job_df = save_jobs_to_excel(jobs_list, filename)
 
    print('{} new job postings retrieved from {}. Stored in {}.'.format(num_listings, 
                                                                          website, filename))
    
    return job_df
        

## Indeed-scraping

<font color='orange'>if customizing this code, you *may* need to change these </font>

### main job scraping fn


links used for testing / figuring out URL structure

```
website = "https://ch.indeed.com/Switzerland-English-Jobs"
base ->
example search https://ch.indeed.com/Stellen?q=Switzerland+English&jt=internship
data job, language = EN, no others: https://ch.indeed.com/Stellen?q=data&lang=en
^ but internship https://ch.indeed.com/Stellen?q=data&jt=internship&lang=en
```

In [15]:
## added by Peter

## indeed switzerland -------------------------------------------------------

def load_indeed_jobs_CH(job_query, job_type=None, language=None, 
                        run_default=False):
    i_website = "https://ch.indeed.com/Stellen?"
    def_website = "https://ch.indeed.com/Stellen?q=Switzerland+English&jt=internship"
    if run_default:
        # switzerland has a unique page shown below, can run by default
        # website = "https://ch.indeed.com/Switzerland-English-Jobs"

        getVars = {'fromage' : 'last', "limit": '50', 'sort' : 'date'}

        url = (def_website + urllib.parse.urlencode(getVars))
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        job_soup = soup.find(id="resultsCol")
    else:
        getVars = {'q' : job_query, 'jt' : job_type, 'lang' : language,
                'fromage' : 'last', "limit": '50', 'sort' : 'date'}

        # if values are not specified, then remove them from the dict (and URL)
        if job_query is None:
            del getVars['q']
        if job_type is None:
            del getVars['jt']
        if language is None:
            del getVars['lang']

        url = (i_website + urllib.parse.urlencode(getVars))
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        job_soup = soup.find(id="resultsCol")

    # return the job soup

    soup_results = {
        "job_soup": job_soup,
        "query_URL": url

    }
    return soup_results




### helper / parsing functions


In [16]:
import pprint as pp

def_URL = "https://ch.indeed.com/Stellen?" + "ADD_queries_here" 

def extract_job_information_indeedCH(job_soup, desired_characs, uURL=def_URL, 
                                     verbose=False, print_all=False):
    # job_elems = job_soup.find_all('div', class_='mosaic-zone-jobcards')
    job_elems = job_soup.find_all('div', class_="job_seen_beacon")

    if print_all:
        print("\nAll found 'job elements' are as follows: \n")
        pp.pprint(job_elems, compact=True)
    
    with open('job_elements.txt', 'w') as f:
        # save to text file for investigation
        print(job_elems, file=f)
    
    cols = []
    extracted_info = []
    
    
    if 'titles' in desired_characs:
        titles = []
        cols.append('titles')
        for job_elem in job_elems:
            titles.append(extract_job_title_indeed(job_elem,
                                                   verbose=verbose))
        extracted_info.append(titles)                    
    
    if 'companies' in desired_characs:
        companies = []
        cols.append('companies')
        for job_elem in job_elems:
            companies.append(extract_company_indeed(job_elem))
        extracted_info.append(companies)
    
    if 'date_listed' in desired_characs:
        dates = []
        cols.append('date_listed')
        for job_elem in job_elems:
            dates.append(extract_date_indeed(job_elem))
        extracted_info.append(dates)
    
    if 'summary' in desired_characs:
        summaries = []
        cols.append('summary')
        for job_elem in job_elems:
            summaries.append(extract_summary_indeed(job_elem))
        extracted_info.append(summaries)

    if 'links' in desired_characs:
        links = []
        cols.append('links')
        for job_elem in job_elems:
            links.append(extract_link_indeedCH(job_elem, uURL))
        extracted_info.append(links)
    
    jobs_list = {}
    
    for j in range(len(cols)):
        jobs_list[cols[j]] = extracted_info[j]
    
    num_listings = len(extracted_info[0])
    
    return jobs_list, num_listings


def extract_job_title_indeed(job_elem, verbose=False):
    title_elem = job_elem.select_one("span[title]").text
    if verbose: print(title_elem)
    try:
        title = title_elem.strip()
    except:
        title="no title"
    return title

def extract_company_indeed(job_elem):
    company_elem = job_elem.find('span', class_='companyName')
    company = company_elem.text.strip()
    return company

def extract_link_indeedCH(job_elem, uURL):
    # some manual shenanigans occur here
    # working example https://ch.indeed.com/Stellen?q=data&jt=internship&lang=en&vjk=49ed864bd5e422fb

    link = job_elem.find('a')['href']
    uURL_list = uURL.split("&fromage=last")
    link = uURL_list[0] + "&" + link
    # replace some text so that the link has a virtual job key. Found via trial and error
    return link.replace("/rc/clk?jk=", "vjk=")

def extract_date_indeed(job_elem):
    date_elem = job_elem.find('span', class_='date')
    date = date_elem.text.strip()
    return date

def extract_summary_indeed(job_elem):
    summary_elem = job_elem.find('div', class_='job-snippet')
    summary = summary_elem.text.strip()
    return summary



### postprocess

1. input dataframe of scraped jobs from indeed
2. ???
3. output sexy_df

In [17]:
from google.colab import files
from datetime import datetime

def indeed_postprocess(i_df, query_term, query_jobtype, verbose=False,
                       shorten_links=False, download_excel=False):
    print("Starting postprocess - ", datetime.now())

    # apply texthero cleaning
    i_df["titles"] = hero.clean(i_df["titles"])
    i_df["summary"] = hero.clean(i_df["summary"])

    # use bit.ly to shorten links
    if shorten_links:
        try:
            len(i_df["short_link"])
            print("found values for short_link, not-recreating")
        except:
            print("no values exist for short_link, creating them now")
            # there is a random delay to not overload APIs, max rt is 5s * num_rows
            i_df["short_link"] = i_df["links"].apply(shorten_URL_bitly) 
    else:
        i_df["short_link"] = "not_created"

    # save file to excel
    rn = datetime.now()
    i_PP_date = rn.strftime("_%m.%d.%Y-%H-%M_")
    i_df["date_pulled"] = rn.strftime("%m.%d.%Y")
    i_df["time_pulled"] = rn.strftime("%H:%M:%S")
    out_name = "JS_DB_" + "query=[term(s)=" + query_term + ", type=" + query_jobtype + "]" + i_PP_date + ".xlsx"
    i_df.to_excel(out_name)
    if verbose: print("Saved {} - ".format(out_name), datetime.now())

    # download if requested
    if download_excel:
        files.download(out_name)
        if verbose: print("Downloaded {} - ".format(out_name), datetime.now())


    return i_df

quick printout of the datatable

In [18]:
def indeed_datatable(i_df, count_what="companies", freq_n=10):
    # basically just wrote this to reduce code down below
    # depends on the colab data_table.DataTable()
    
    print("Count of column '{}' appearances in search:\n".format(count_what))
    comp_list_1 = i_df[count_what].value_counts()
    pp.pprint(comp_list_1.head(freq_n), compact=True)  

    i_df_disp = i_df.copy()
    i_df_disp["summary_short"] = i_df_disp["summary"].apply(text_first_N)
    i_df_disp.drop(columns=["links", "summary"], inplace=True) # drop verbose columns

    return i_df_disp 

    # i_df_disp for use inside 
    # data_table.DataTable(i_df_disp, 
                        #  include_index=False, num_rows_per_page=table_n)


---

# Main

## parameters

## key settings

In [19]:
# define whether or not to shorten links
shorten_key = False #@param {type:"boolean"}
# define whether or not to download excel versions of the files
# download_key = for specific searches deemed relevant
# download_all includes all searches
download_key = True #@param {type:"boolean"}
download_all = False #@param {type:"boolean"}

# determines columns in output dataframe post-scraping
desired_characs = ['titles', 'companies', 'links', 
                   'date_listed', 'summary']

### set and load models

In [20]:
using_google_USE = True #@param {type:"boolean"}
using_gensim_w2v = False #@param {type:"boolean"}

# only load models declared as used

if using_google_USE:
    meine_embeddings = load_google_USE()

if using_gensim_w2v:
    w2v_model = load_gensim_word2vec()

loaded google USE embeddings in 0.45 minutes


## Swiss Jobs - Indeed

```
This function extracts all the desired characteristics of all new job postings
    of the title and location specified and returns them in single file.
The arguments it takes are:

    - Website: to specify which website to search 
        - (options: 'indeed' or 'indeed_default')
    - job_query: words that you want to narrow down the jobs to. 
        - for example 'data'
    - job_type: 
        - 'internship' or 'fulltime' or 'permanent'
    - language: 
        - 'en' or 'de' or other languages.. 'fr'? ew
    - Desired_characs: what columns of data do you want to extract? options are:
        - 'titles', 'companies', 'links', 'date_listed', 'summary'
    - Filename: default is "JS_test_results.xls", can be changed to whatever 
```


### query 1 - internship in "data"

In [21]:
jq1 = "data" #@param {type:"string"}
jt1 = "internship" #@param {type:"string"}
lan = "en" #@param {type:"string"}


In [22]:
from google.colab import data_table

# variables for fn defined in form above

chdf1 = find_CHjobs_from(website="indeed", desired_characs=desired_characs, 
                         job_query=jq1, job_type=jt1, language=lan)

q1_processed = indeed_postprocess(chdf1, query_term=jq1, query_jobtype=jt1,
                       shorten_links=shorten_key, download_excel=download_key)

data_table.DataTable(indeed_datatable(q1_processed), 
                     include_index=False, num_rows_per_page=20)

50 new job postings retrieved from indeed. Stored in Jul-08-2021_[raw]_scraped_jobs_CH.xls.
Starting postprocess -  2021-07-08 03:49:55.055999


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Count of column 'companies' appearances in search:

EY                                             6
Ernst & Young                                  3
United Nations                                 3
Novartis                                       3
IWC                                            3
International Telecommunication Union (ITU)    2
On AG                                          2
HILTI                                          2
Lunaphore Technologies                         2
swisspuls                                      1
Name: companies, dtype: int64


,titles,companies,date_listed,short_link,date_pulled,time_pulled,summary_short
0,manager financial services risk emeia financia...,EY,Gerade geschaltet,not_created,07.08.2021,03:49:55,encouraged investigate new risk areas ex..
1,intern finance peopleops strategy crypto startup,CasperLabs,Gerade geschaltet,not_created,07.08.2021,03:49:55,compile financial operational data repor..
2,intern production clothing men classic f,Holy Fashion Group,Heute,not_created,07.08.2021,03:49:55,fascinating role rounded preparing seaso..
3,intern thesis student - mechatronic mechanical...,HILTI,Heute,not_created,07.08.2021,03:49:55,matlab c coding basics plotting data man..
4,corporate finance analyst trainee permanent po...,CreditGate24/Advanon,Heute,not_created,07.08.2021,03:49:55,support data analytics stream analyzing ..
5,intern data scientist,Swiss Re,Heute,not_created,07.08.2021,03:49:55,support developing impactful exciting da..
6,material intern apparel,On AG,Heute,not_created,07.08.2021,03:49:55,help collate information material specif..
7,training interniship,LVMH,Heute,not_created,07.08.2021,03:49:55,new collaborators accounts creation data..
8,career internship advisor,Sommet Education,Heute,not_created,07.08.2021,03:49:55,ad hoc administrative tasks data collect..
9,legal trainee 6months,Swissquote,Heute,not_created,07.08.2021,03:49:55,supporting team number different tasks r..


**Viz Query 1**

In [23]:
viz1_df = q1_processed.copy()
viz1_df.drop(columns=["links", "short_link"], inplace=True)

if using_google_USE:

    # general rule - if # of jobs returned > 25 may want to turn off text in 
    # one or both plots (summary text and title text)

    vizjobs_googleUSE(viz1_df, "summary", meine_embeddings, 
                      save_plot=True, show_text=True,
                        query_name=jt1 + " in " + jq1, viz_type="pca")
    
    vizjobs_googleUSE(viz1_df, "titles", meine_embeddings, 
                      save_plot=True, show_text=False,
                        query_name=jt1 + " in " + jq1, viz_type="pca")
else:
    viz_job_data_word2vec(viz1_df, "summary", save_plot=True, h=720,
                          query_name=jt1 + " in " + jq1, viz_type="pca")
    viz_job_data_word2vec(viz1_df, "titles", save_plot=True, h=720,
                          query_name=jt1 + " in " + jq1, viz_type="pca")




For google_USE-internship in data: opt. # of k-means clusters is 9 



plot generated -  2021-07-08 03:49:59.667419

For google_USE-internship in data: opt. # of k-means clusters is 10 



plot generated -  2021-07-08 03:50:01.193124


### query 2 - all jobs in Switzerland for English Speakers

*This is the special case where Indeed shows you all CH-English jobs and it doesn't follow standard search query logic / HTML (has its own page)*

In [24]:
jq2 = "indeed_default" # passing this phrase in causes it to search for all en jobs
jt2 = "all"
# in the case of "run the special case on Indeed" query terms don't matter
chdf2 = find_CHjobs_from(website="indeed_default", job_query="gimme", 
                         desired_characs=desired_characs)

q2_processed = indeed_postprocess(chdf2, query_term=jq2, query_jobtype=jt2,
                       shorten_links=False, download_excel=False)

data_table.DataTable(indeed_datatable(q2_processed), 
                     include_index=False, num_rows_per_page=20)

50 new job postings retrieved from indeed_default. Stored in Jul-08-2021_[raw]_scraped_jobs_CH.xls.
Starting postprocess -  2021-07-08 03:50:03.481620
Count of column 'companies' appearances in search:

MDPI AG                            4
ABB                                4
Philochem AG                       3
H+O communications Ltd.            3
Paul Scherrer Institut Villigen    2
Randstad                           2
World Health Organization          2
digitalswitzerland                 1
ITech Consult                      1
Roche                              1
Name: companies, dtype: int64


,titles,companies,date_listed,short_link,date_pulled,time_pulled,summary_short
0,client service executive,BNY Mellon,Gerade geschaltet,not_created,07.08.2021,03:50:03,fulfill administrative needs fund buying..
1,sale associate landquart,Groupe Rossignol,Gerade geschaltet,not_created,07.08.2021,03:50:03,rossignol group looking sales associate ..
2,ux ui designlead onelog,TX Group,Gerade geschaltet,not_created,07.08.2021,03:50:03,employment level location zurich mobile ..
3,support training application specialist molecu...,Roche,Gerade geschaltet,not_created,07.08.2021,03:50:03,global customer support global organizat..
4,senior gl accountant temporary,Exact Sciences Corporation,Gerade geschaltet,not_created,07.08.2021,03:50:03,senior gl accountant sga directly respon..
5,consultant methods standards projects,World Health Organization,Gerade geschaltet,not_created,07.08.2021,03:50:03,contract duration years months days mont..
6,senior project manager renewable energy therma...,AFRY,Gerade geschaltet,not_created,07.08.2021,03:50:03,develop commercially lead project execut..
7,digital project coordinator,H+O communications Ltd.,Gerade geschaltet,not_created,07.08.2021,03:50:03,support initial digital concept implemen..
8,multimedia producer web specialist,H+O communications Ltd.,Gerade geschaltet,not_created,07.08.2021,03:50:03,develop wide range printed online articl..
9,internship - medical communications,H+O communications Ltd.,Gerade geschaltet,not_created,07.08.2021,03:50:03,medical communications pharmaceutical ma..


**Viz Query 2**

In [25]:
viz_q2 = q2_processed.copy()
viz_q2.drop(columns="links", inplace=True)

if using_google_USE:

    # general rule - if # of jobs returned > 25 may want to turn off text in 
    # one or both plots (summary text and title text)

    vizjobs_googleUSE(viz_q2, "summary", meine_embeddings, 
                      save_plot=True, show_text=True,
                        query_name="all listings for CH eng. jobs", viz_type="tsne")
    
    vizjobs_googleUSE(viz_q2, "titles", meine_embeddings, 
                      save_plot=True, show_text=False,
                        query_name="all listings for CH eng. jobs", viz_type="tsne")
else:
    viz_job_data_word2vec(viz_q2, "summary", save_plot=False, h=720,
                          query_name="all listings for CH eng. jobs", viz_type="tsne")
    viz_job_data_word2vec(viz_q2, "titles", save_plot=False, h=720,
                          query_name="all listings for CH eng. jobs", viz_type="tsne")


For google_USE-all listings for CH eng. jobs: opt. # of k-means clusters is 7 



plot generated -  2021-07-08 03:50:05.180987


/usr/local/lib/python3.7/dist-packages/kneed/knee_locator.py:304: UserWarning:

No knee/elbow found



Warning - google_USE-all listings for CH eng. jobs has no solution for optimal k-means
Returning # of clusters as max allowed ( 15 )


plot generated -  2021-07-08 03:50:07.018735
